In [333]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from time import sleep
from collections import Counter

pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set()

In [334]:
def generaPath(tipo, nombre):
    prepath = str(os.getcwd())
    #print(prepath)
    if (tipo == "I"):
        path = prepath + '\\origen\\' + nombre
    else :
        path = prepath + '\\generados\\' + nombre
    return path

In [335]:
df_cq_tr = pd.read_csv(generaPath('O', 'df_cq_tr.csv'))
df_cq_re = pd.read_csv(generaPath('O', 'df_cq_re.csv'))

df_cc_tr = pd.read_csv(generaPath('O', 'df_cc_tr.csv'))
df_cc_re = pd.read_csv(generaPath('O', 'df_cc_re.csv'))

df_ct_tr = pd.read_csv(generaPath('O', 'df_ct_tr.csv'))
df_ct_re = pd.read_csv(generaPath('O', 'df_ct_re.csv'))

df_cd_tr = pd.read_csv(generaPath('O', 'df_cd_tr.csv'))
df_cd_re = pd.read_csv(generaPath('O', 'df_cd_re.csv'))

df_cs_tr = pd.read_csv(generaPath('O', 'df_cs_tr.csv'))
df_cs_re = pd.read_csv(generaPath('O', 'df_cs_re.csv'))

df_cs_tr_sb = pd.read_csv(generaPath('O', 'df_cs_tr_sb.csv'))
df_cs_re_sb = pd.read_csv(generaPath('O', 'df_cs_re_sb.csv'))


df_sel_sb_tr = pd.read_csv(generaPath('O','df_sel_sb_tr.csv'))
df_sel_sb_re = pd.read_csv(generaPath('O','df_sel_sb_re.csv'))

df_sel_sb_re = pd.read_csv(generaPath('O','df_sel_si_tr.csv'))
df_sel_si_re = pd.read_csv(generaPath('O','df_sel_si_re.csv') )

sbs = pd.read_csv(generaPath('O','sbs.csv'))
singles = pd.read_csv(generaPath('O','singles.csv'))

df_tr_fu_filt = pd.read_csv(generaPath('O', 'df_tr_fu_filt_delta.csv'))
df_re_fu_filt = pd.read_csv(generaPath('O', 'df_re_fu_filt_delta.csv'))

df = pd.read_csv(generaPath('I', 'expdata2.csv'), parse_dates=['fecha'])

In [336]:
#df

In [337]:
df_tr_fu_filt



,idSorteo,n1,n2,n3,n4,n5,sb,delta
0,2140,17,25,29,34,36,12,HDEBW
1,2139,12,15,22,26,28,7,CGDBZ
2,2138,6,7,9,18,35,6,ABIPN
3,2137,1,6,16,22,31,16,EJFIM
4,2136,18,35,37,39,42,13,PBBCR
...,...,...,...,...,...,...,...,...
465,1675,1,20,36,39,43,10,ROCDA
466,1674,5,6,7,12,37,6,AAEXK
467,1673,6,19,25,28,36,1,MFCHM
468,1672,1,2,25,31,33,14,AVFBK


In [338]:
df_re_fu_filt

,idSorteo,n1,n2,n3,n4,n5,sb,delta
0,2140,4,7,19,33,39,8,CLNFH
1,2139,1,4,12,13,33,13,CHASK
2,2138,1,11,22,32,33,12,JKJAK
3,2137,11,12,28,29,32,13,AOACU
4,2136,5,7,23,34,43,11,BOKIE
...,...,...,...,...,...,...,...,...
465,1675,3,5,8,20,24,14,BCLDU
466,1674,5,12,15,20,42,14,GCEUF
467,1673,8,21,22,37,38,4,MAÑAM
468,1672,9,12,16,28,38,1,CDLJN


Filtra los numeros singles repetidos

In [339]:
def cantidad_no_rep():
    l= singles.sort_values("tupla")
    l2 = l.drop('R', 1)
    l3 = l2.drop_duplicates()
    print(l3)
    return l3

In [340]:
def validar_quinteto_list():
    l= singles.sort_values("tupla")
    l2 = l.drop('R', 1)
    l3 = l2.drop_duplicates()
    return l3

In [341]:
# def unsplited_tupla(secuence):
#     num = str(secuence[0])
#     partido = num.split(sep='|') if (num.find('|') >= 1) else num
#     return partido

In [342]:
def secuencelist_unbuilder(secuence):
    num = str(secuence[0])
    partido = num.split(sep='|') if (num.find('|') >= 1) else num
    return partido

In [343]:
def crea_sorteos(single,sb, secuence):
    sorteo = []
    enteros = []
    # splitedtupla = unsplited_tupla(secuence)
    #enteros = secuencelist_unbuilder(str(secuence[0]))
    # print(splitedtupla)
    print(" Tipo Enteros")
    print(type(enteros))
    print(enteros)
    #sorteo = singles.iloc[enteros, 0]
    #print(sorteo)
    #sorteo.append(str(sb))
    return sorteo

In [344]:
def secuencelist_builder(a,b,c,d,e):
    secuencia=[]
    secuencia.append(a)
    secuencia.append(b)
    secuencia.append(c) 
    secuencia.append(d)
    secuencia.append(e)
    palabra = '|'.join([str(et) for et in secuencia])
    return palabra, secuencia


In [345]:
def crea_secuencias(totalconteo):
    secuencias = []
    secuence = []
    lsecuence =[]
    secuencia = []
    i = 0
    a = i
    b = a + 1
    c = b + 1
    d = c + 1
    e = d + 1
    while (a <= totalconteo):
        while(b <= totalconteo):
            while(c <= totalconteo):
                while(d <= totalconteo):
                    while(e <= totalconteo):
                        secuencia, secuence = secuencelist_builder(a,b,c,d,e)
                        secuencias.append(secuencia)
                        lsecuence.append(secuence)
                        e += 1
                    d += 1
                    e = d + 1
                c += 1
                d = c + 1
                e = d + 1
            b += 1
            c = b + 1
            d = c + 1
            e = d + 1
        a = a + 1
        b = a + 1
        c = b + 1
        d = c + 1 
        e = d + 1
    return secuencias, lsecuence


Construye los numeros candidatos de acuerdo a los numeros que salieron en singles y sbs, itera para cada superbalota no esta usando el contenido de las superbalotas

In [346]:
def combinations_generator(sbs, l):
    sorteos = []
    lsecuence = []  
    secuencias = []
    cant = len(l) - 1
    secuencias, lsecuence = crea_secuencias(cant)
    cantsecuence = len(lsecuence)
    print("cant:" + str(cant) + "  cantsecuence: " + str(cantsecuence))
    # for index, row in sbs.iterrows():
        # sb = str(row[0])
    i = 0
    for rows in secuencias:
        lsec = lsecuence[int(i)]
        sorteo = l3.iloc[lsec, 0]
        i = i + 1
        sorteos.append(sorteo.to_list())
    dft = sorteos
    # print(dft)
    return dft

In [347]:
def combinations_getter_saver():
    secus = []
    l3 = cantidad_no_rep()
    print("valor de l3")
    print(len(l3))
    print(singles)
    print(sbs)
    dft_sorteos = combinations_generator(sbs, l3)
    dft = pd.DataFrame (dft_sorteos, columns = ['n1','n2','n3','n4','n5'])
    dft.to_csv(generaPath('O','dft.csv'), header=['n1','n2','n3','n4','n5'], index=None, sep=',', mode='w' )
    #print(dft_sorteos)
    return dft

In [348]:
#cantidad_no_rep()

In [349]:
#combinations_getter_saver()

In [350]:
def construir_trio(n1,n2,n3):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3)
    return salida

In [351]:
def construir_list_trio(n1,n2,n3):
    salida = [n1,n2,n3]
    # salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    #print(salida)
    return salida

In [352]:
def construir_cuarteto(n1,n2,n3,n4):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4)
    return salida

In [353]:
def construir_list_cuarteto(n1,n2,n3,n4):
    salida = [n1,n2,n3,n4]
    # salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    # print("salida de  cuarteto")
    # print(salida)
    return salida

In [354]:
def construir_quinteto(n1,n2,n3,n4,n5):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    return salida

In [355]:
def construir_list_quinteto(n1,n2,n3,n4,n5):
    salida = [n1,n2,n3,n4,n5]
    # salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    # print(type(salida))
    return salida

In [356]:
def all_trios(n1,n2,n3,n4,n5):
    trio01 = construir_list_trio(n1,n2,n3)
    trio02 = construir_list_trio(n1,n2,n4)
    trio03 = construir_list_trio(n1,n2,n5)
    trio04 = construir_list_trio(n2,n3,n4)
    trio05 = construir_list_trio(n2,n3,n5)
    trio06 = construir_list_trio(n2,n4,n5)
    trio07 = construir_list_trio(n3,n4,n5)
    trio08 = construir_list_trio(n1,n3,n4)
    trio09 = construir_list_trio(n1,n3,n5)
    trio10 = construir_list_trio(n1,n4,n5)
    return trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10

In [357]:
def all_cuartetos(n1,n2,n3,n4,n5):
    cuarteto1 = construir_list_cuarteto(n1, n2, n3, n4)
    cuarteto2 = construir_list_cuarteto(n1, n2, n3, n5)
    cuarteto3 = construir_list_cuarteto(n1, n2, n4, n5)
    cuarteto4 = construir_list_cuarteto(n1, n3, n4, n5)
    cuarteto5 = construir_list_cuarteto(n2, n3, n4, n5)
    return cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5

In [358]:
def find_previous_coincidence(n1,n2,n3,n4,n5, df):
    df_found = df.loc[(df.n1 == n1) & (df.n2 == n2) & (df.n3 == n3) & (df.n4 == n4) & (df.n5 == n5)]
    print(df_found)
    encontrados = 0
    total_rows = int(df_found.shape[0])
    if total_rows > 0:
        encontrados = 1
        return str(df_found) , encontrados
    else:
        return str(df_found),  encontrados

In [359]:
def find_previous_coincidence_c(n1,n2,n3,n4, df):
    df_found = df.loc[(df.n1 == n1) & (df.n2 == n2) & (df.n3 == n3) & (df.n4 == n4)]
    print(df_found)
    encontrados = 0
    total_rows = int(df_found.shape[0])
    if total_rows > 0:
        encontrados = 1
        return str(df_found) , encontrados
    else:
        return str(df_found),  encontrados

In [360]:
def find_previous_coincidence_t(n1,n2,n3, df):
    df_found = df.loc[(df.n1 == n1) & (df.n2 == n2) & (df.n3 == n3) ]  #& (df.n4 == n4) & (df.n5 == n5)
    print(df_found)
    encontrados = 0
    total_rows = int(df_found.shape[0])
    if total_rows > 0:
        encontrados = 1
        return str(df_found) , encontrados
    else:
        return str(df_found),  encontrados

In [361]:
def validar_sumar_resultados_sorteos(n1,n2,n3,n4,n5, df_to_search):
    mensaje, suma = find_previous_coincidence(n1,n2,n3,n4,n5, df_to_search)
    # print(suma)
    return mensaje, suma

In [362]:
def validar_sumar_resultados_sorteos_c(n1,n2,n3,n4, df_to_search):
    mensaje, suma = find_previous_coincidence_c(n1,n2,n3,n4, df_to_search)
    # print(suma)
    return mensaje, suma

In [363]:
def validar_sumar_resultados_sorteos_t(n1,n2,n3,df_to_search):
    mensaje, suma = find_previous_coincidence_t(n1,n2,n3,df_to_search)
    # print(suma)
    return mensaje, suma

In [364]:
def ingresar_sorteo_validar_q(n1,n2,n3,n4,n5, quint):
    # quinteto = construir_quinteto(n1, n2, n3, n4, n5)
    quinteto = construir_list_quinteto(n1, n2, n3, n4, n5)
    quint.append(quinteto)
    return quinteto


In [365]:
def ingresar_sorteo_validar_c(n1,n2,n3,n4,n5, cuart):
    c1, c2, c3, c4, c5 = all_cuartetos(n1, n2, n3, n4, n5)
    cuart.append(c1)
    cuart.append(c2)
    cuart.append(c3)
    cuart.append(c4)
    cuart.append(c5)
    return cuart

In [366]:
def ingresar_sorteo_validar_t(n1,n2,n3,n4,n5, trios):
    t01, t02, t03, t04, t05, t06, t07, t08, t09, t10 = all_trios(n1, n2, n3, n4, n5)
    trios.append(t01)
    trios.append(t02)
    trios.append(t03)
    trios.append(t04)
    trios.append(t05)
    trios.append(t06)
    trios.append(t07)
    trios.append(t08)
    trios.append(t09)
    trios.append(t10)
    return trios

In [367]:
def sumar_tuplas(n1,n2,n3,n4,n5,df_exc):   
    rechazados = df_exc.loc[(df_exc.n1 == n1) & (df_exc.n2 == n2) & (df_exc.n3 == n3) & (df_exc.n4 == n4) & (df_exc.n5 == n5)]
    elegidos = df_exc.loc[(df_exc.n1 != n1) | (df_exc.n2 != n2) | (df_exc.n3 != n3) | (df_exc.n4 != n4) | (df_exc.n5 != n5  )]
    cant = len(rechazados)
    cant2 = len(elegidos)
    print("Cant Rechazados: " + str(cant) + " Cant Elegidos:" + str(cant2))
    return elegidos, rechazados, cant

In [368]:
def sumar_tuplas_c(n1,n2,n3,n4,df_exc):   
    rechazados = df_exc.loc[(df_exc.n1 == n1) & (df_exc.n2 == n2) & (df_exc.n3 == n3) & (df_exc.n4 == n4) ] #& (df_exc.n5 == n5)
    elegidos = df_exc.loc[(df_exc.n1 != n1) | (df_exc.n2 != n2) | (df_exc.n3 != n3) | (df_exc.n4 != n4) ] # | (df_exc.n5 != n5  )
    cant = len(rechazados)
    return elegidos, rechazados, cant

In [369]:
def sumar_tuplas_t(n1,n2,n3,df_exc):   
    rechazados = df_exc.loc[(df_exc.n1 == n1) & (df_exc.n2 == n2) & (df_exc.n3 == n3) ] #& (df_exc.n4 == n4) & (df_exc.n5 == n5)
    elegidos = df_exc.loc[(df_exc.n1 != n1) | (df_exc.n2 != n2) | (df_exc.n3 != n3) ] #| (df_exc.n4 != n4) | (df_exc.n5 != n5  )
    cant = len(rechazados)
    return elegidos, rechazados, cant

In [370]:
def couples_builder(dft):
    i = 0
    dictionary = {}
    can_elegidos = len(dft)
    print(can_elegidos)
    validos=[]
    quint=[]
    cuart=[]
    trios=[]
    for index, row in dft.iterrows():
        sbs = (row)
        n1 = int(sbs[0])
        n2 = int(sbs[1])
        n3 = int(sbs[2])
        n4 = int(sbs[3])
        n5 = int(sbs[4]) 
        porcentaje = ((i+1)/can_elegidos)*100
        totaltrios = 0
        totalcuartetos = 0
        totalquintetos = 0
        quint = ingresar_sorteo_validar_q(n1, n2, n3, n4, n5, quint)
        cuart = ingresar_sorteo_validar_c(n1, n2, n3, n4, n5, cuart)
        trios = ingresar_sorteo_validar_t(n1, n2, n3, n4, n5, trios)
        totales = 0
        totaltrios = 0
        i = i + 1
    print(len(quint))
    print(len(cuart))
    print(len(trios))
    return quint, cuart, trios

In [371]:
def validator_t(trios):
    trios_df = pd.DataFrame (cuart, columns = ['n1','n2','n3','n4'])
    validar_sumar_resultados_sorteos_t(n1,n2,n3,trios_df)
    elegidos_t, rechazados, totaltrios  = sumar_tuplas_t(n1,n2,n3,df_excluidos)
    if (totaltrios <= 4):
        choosen.append(row)
        totales = (totalquintetos/ 2) + (totalcuartetos/10) + (totaltrios/20)
        print("total quintetos: " + str(totalquintetos) + ", total cuartetos: " + str(totalcuartetos) + ", total trios: " + str(totaltrios))
        print("totales: "+ str(totales))
        print("---------")
    else: 
        rejected.append(row)

In [372]:
def validator_c(cuart, trios):
    cuart_df = pd.DataFrame (cuart, columns = ['n1','n2','n3','n4'])
    validar_sumar_resultados_sorteos_c(n1,n2,n3,n4, cuart_df)
    elegidos_c, rechazados, totalcuartetos  = sumar_tuplas_c(n1,n2,n3,n4,df_excluidos) #, df_cc_tr, df_cc_re)
    if (totalcuartetos == 0):
        validator_t(trios)
    else:
        rejected.append(row)

In [373]:
def validator(quint, cuart, trios, dft ):
    i = 0
    dictionary = {}
    can_elegidos = len(dft)
    #can_elegidos = 1000
    print(can_elegidos)
    validos=[]
    choosen = []
    rejected = []
    rejecteds = []
    df_excluidos = []
    df_excluidos = df_tr_fu_filt
    iteracion = 0
    for index, row in dft.iterrows():
        sbs = (row)
        # print(type(sbs[0]))
        n1 = int(sbs[0])
        n2 = int(sbs[1])
        n3 = int(sbs[2])
        n4 = int(sbs[3])
        n5 = int(sbs[4]) 
        porcentaje = ((i+1)/can_elegidos)*100
        print("Iteracion " + str(i) + " Porcentaje Avance: "+ str(porcentaje), " N1:" + str(n1), " N2:" + str(n2), " N3:" + str(n3), " N4:" + str(n4), " N5:" + str(n5))
        totaltrios = 0
        totalcuartetos = 0
        totalquintetos = 0
        choosen, rejected, totalquintetos  = sumar_tuplas(n1,n2,n3,n4,n5, df_excluidos)
        iteracion = iteracion + 1
        if (totalquintetos == 0):
            #validator_c(cuart, trios)
            print("total Quintetos == 0")
            print(rejected)
            print(totalquintetos)
        else:
            print("total Quintetos != 0")
            rejecteds.append(rejected)
        if (iteracion >= can_elegidos):
            salir = int(input("ingresa si quieres seguir evaluando total quintetos > 0: "))
            if(salir > 0 ):
                return choosen, rejected
        totales = 0
        totaltrios = 0
        i = i + 1
    return choosen, rejected


In [374]:
candidates = []
rejected =[]
# dft=[]
dft = combinations_getter_saver()


    tupla
0      13
8      14
7      16
1      17
11     18
10     21
2      25
3      29
9      31
4      34
12     35
5      36
6      43
valor de l3
13
    tupla  R
0      13  0
1      17  1
2      25  1
3      29  1
4      34  1
5      36  2
6      43  1
7      16  1
8      14  1
9      31  1
10     21  1
11     18  1
12     35  2
   tupla  R
0     10  1
1      3  2
2      4  2
3     11  3
4      1  2
5      9  2
6     10  4
7     16  4
cant:12  cantsecuence: 1287


In [375]:
quint, cuart, trios = couples_builder(dft)
#choosen, rejected = validator(quint, cuart, trios, candidates, rejected)
print(type(quint))


1287
5
6435
12870
<class 'list'>


In [376]:
choosen = []
rejected = []
choosen, rejected = validator(quint, cuart, trios, dft)

1287
Iteracion 0 Porcentaje Avance: 0.0777000777000777  N1:2  N2:3  N3:6  N4:9  N5:11
Cant Rechazados: 0 Cant Elegidos:470
total Quintetos == 0
Empty DataFrame
Columns: [idSorteo, n1, n2, n3, n4, n5, sb, delta]
Index: []
0
Iteracion 1 Porcentaje Avance: 0.1554001554001554  N1:2  N2:3  N3:6  N4:9  N5:14
Cant Rechazados: 0 Cant Elegidos:470
total Quintetos == 0
Empty DataFrame
Columns: [idSorteo, n1, n2, n3, n4, n5, sb, delta]
Index: []
0
Iteracion 2 Porcentaje Avance: 0.2331002331002331  N1:2  N2:3  N3:6  N4:9  N5:16
Cant Rechazados: 0 Cant Elegidos:470
total Quintetos == 0
Empty DataFrame
Columns: [idSorteo, n1, n2, n3, n4, n5, sb, delta]
Index: []
0
Iteracion 3 Porcentaje Avance: 0.3108003108003108  N1:2  N2:3  N3:6  N4:9  N5:17
Cant Rechazados: 0 Cant Elegidos:470
total Quintetos == 0
Empty DataFrame
Columns: [idSorteo, n1, n2, n3, n4, n5, sb, delta]
Index: []
0
Iteracion 4 Porcentaje Avance: 0.3885003885003885  N1:2  N2:3  N3:6  N4:9  N5:18
Cant Rechazados: 0 Cant Elegidos:470
total

In [377]:
len(choosen)

470

In [378]:

len(rejected)

0

In [379]:
# import ipykernel  
# ipykernel.__version__


In [380]:
# import sys
# sys.version